In [3]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tqdm import tqdm

dataset_path = r"D:\DESKTOP\AI proj\gender_dataset_2"


In [4]:
images = []
labels = []

class_names = sorted(os.listdir(dataset_path))
label_map = {name: idx for idx, name in enumerate(class_names)}

print(label_map)

{'female': 0, 'male': 1}


In [5]:
for label_name in class_names:
    folder_path = os.path.join(dataset_path, label_name)
    if not os.path.isdir(folder_path):
        continue
    for filename in tqdm(os.listdir(folder_path), desc=f"Loading {label_name}"):
        img_path = os.path.join(folder_path, filename)
        try:
            img = load_img(img_path, target_size=(128, 128))
            img_array = img_to_array(img) / 255.0  # normalize
            images.append(img_array)
            labels.append(label_map[label_name])
        except Exception as e:
            print(f"Skipped {img_path}: {e}")

X = np.array(images)
y = np.array(labels)

Loading male: 100%|██████████| 12500/12500 [00:48<00:00, 256.00it/s]


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, stratify=y, random_state=0)

In [7]:
from keras.layers import Input

In [8]:
model = Sequential([
    Input(shape=(128, 128, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [9]:
training = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

Epoch 1/10
524/524 [==============================] - 477s 891ms/step - loss: 0.4309 - accuracy: 0.7988 - val_loss: 0.3331 - val_accuracy: 0.8533
Epoch 2/10
524/524 [==============================] - 272s 518ms/step - loss: 0.3056 - accuracy: 0.8614 - val_loss: 0.3074 - val_accuracy: 0.8640
Epoch 3/10
524/524 [==============================] - 245s 468ms/step - loss: 0.2640 - accuracy: 0.8862 - val_loss: 0.2924 - val_accuracy: 0.8823
Epoch 4/10
524/524 [==============================] - 199s 380ms/step - loss: 0.2310 - accuracy: 0.9024 - val_loss: 0.2715 - val_accuracy: 0.8846
Epoch 5/10
524/524 [==============================] - 183s 350ms/step - loss: 0.2000 - accuracy: 0.9151 - val_loss: 0.2863 - val_accuracy: 0.8859
Epoch 6/10
524/524 [==============================] - 222s 424ms/step - loss: 0.1769 - accuracy: 0.9270 - val_loss: 0.2948 - val_accuracy: 0.8913
Epoch 7/10
524/524 [==============================] - 236s 450ms/step - loss: 0.1513 - accuracy: 0.9387 - val_loss: 0.3063 -

In [12]:
model_save_path = r"D:\DESKTOP\AI proj\project\models\gender_cnn_model.h5"
model.save(model_save_path)

c:\miniconda\envs\myenv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [11]:
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy (y, y_predicted):", accuracy)

225/225 [==============================] - 34s 149ms/step
Accuracy (y, y_predicted): 0.8861799944274171
